In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd '/content/drive/MyDrive/PyAI'

/content/drive/MyDrive/PyAI


In [9]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [10]:
train = datasets.MNIST(root = "/data", train = True, transform=transforms.ToTensor(), download = True)
test = datasets.MNIST(root = "/data", train = False, transform=transforms.ToTensor(), download = True)

train_data = DataLoader(train, batch_size = 100, shuffle = True)
test_data = DataLoader(test, batch_size = 1000, shuffle = False)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#x, t 입력
#dataloader에서 처리

#함수 정의
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5), #(1*28*28) -> (16*24*24)
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5), #(16*24*24) -> (32*20*20)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32*20*20) -> (32*10*10)
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5), #(32*10*10) -> (64*6*6)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64*6*6) -> (64*3*3)
        )
        self.g = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10),
        )
    def forward(self,x) :
        if x.dim() == 3 :
            x = x.unsqueeze(0)
        x = self.f(x)
        x = x.reshape(x.shape[0], -1)
        x = self.g(x)
        return x
F = NN()
F = F.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.002)
epoch = 10
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_data :
#순전파
        y = F(x.to(device))
#손실함수
        loss = loss_func(y, t.to(device))
        loss_sum += loss
#역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_sum /= len(train_data)

    correct = 0
    total = 0
    for x, t in test_data :
        with torch.no_grad() :
            y = F(x.to(device))
        correct += (y.argmax(dim = 1) == t.to(device)).sum().item()
        total += len(x)
    acc = correct / total

    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc}")
    if (cnt >= 3) or (acc >= 0.99) :
        print("train halted")
        break

epoch 1 | loss 0.17270225286483765 | acc 0.981
epoch 2 | loss 0.04449569806456566 | acc 0.9903
train halted


In [13]:
cnt = 0;
total = 0
err = []

for x, t in test_data :
    y = F(x.to(device))
    for i in range(x.shape[0]) :
        if torch.argmax(y[i]) == t.to(device)[i] :
            cnt += 1
        else :
            err.append(total)
        total += 1

print("correct_posibility : %f" %(cnt / total))
print(err)
torch.save(F.to("cpu"),"MNIST.pt")

correct_posibility : 0.990300
[62, 95, 115, 247, 259, 320, 449, 582, 726, 740, 938, 947, 965, 1014, 1039, 1044, 1112, 1202, 1226, 1247, 1260, 1319, 1393, 1425, 1570, 1621, 1709, 1878, 1901, 2018, 2043, 2129, 2130, 2293, 2329, 2387, 2447, 2454, 2455, 2462, 2597, 2654, 2836, 2896, 2939, 2953, 2995, 3225, 3422, 3503, 3534, 3558, 3559, 3597, 3662, 3726, 3762, 3808, 3906, 3976, 4007, 4265, 4284, 4380, 4620, 4740, 4783, 4807, 5201, 5409, 5450, 5654, 5937, 6172, 6173, 6559, 6560, 6571, 6572, 6576, 6597, 6625, 6632, 6651, 6783, 7235, 7574, 8246, 8382, 8408, 8504, 8527, 9009, 9505, 9664, 9729, 9922]
